<a href="https://colab.research.google.com/github/marvin-hansen/fastai/blob/master/Fast_Predict_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock price prediction with fast.ai





## Mount GDrive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/'
#!cd /content/gdrive/MyDrive/fastai-v3/
!ls

Mounted at /content/gdrive
aclocal.m4	config.status  libtool	    ta-lib
AMZN.csv	config.sub     ltmain.sh    ta-lib-0.4.0-src.tar.gz
autogen.sh	configure      Makefile     ta-lib-config
autom4te.cache	configure.in   Makefile.am  ta-lib-config.in
bin		depcomp        Makefile.in  ta-lib.dpkg
CHANGELOG.TXT	HISTORY.TXT    missing	    ta-lib.dpkg.in
config.guess	include        models	    ta-lib.spec
config.log	install-sh     src	    ta-lib.spec.in


## Constants & Flags

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
#batch size 

bs = 64
# bs = 32 
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart

## Install TA/Lib for Technical Analysis

In [0]:
# install TA LIB for Technical Analysis
# http://ta-lib.org/hdr_dw.html
# https://github.com/mrjbq7/ta-lib
# https://stackoverflow.com/questions/49648391/how-to-install-ta-lib-in-google-colab
!mkdir ta-lib 
!cd ta-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib


## Imports


In [0]:
import fastai
from fastai import *
from fastai.basics import *
from fastai.tabular import *
from fastai.metrics import error_rate

import matplotlib.pyplot as plt
from PIL import Image

print("FastAI Version: " + str(fastai.__version__))


FastAI Version: 1.0.46


In [0]:
import torch

print("PyTorch Version: " + str(torch.__version__))

PyTorch Version: 1.0.1.post2


In [0]:
import talib

print("TA Lib Version: " + str(talib.__version__))

TA Lib Version: 0.4.17


## Verify GPU *acceleration*

In [0]:
torch.cuda.current_device()

print("Cude available: " + str(torch.cuda.is_available()))
print("Cude enabled:" + str(torch.backends.cudnn.enabled))

#https://stackoverflow.com/questions/48152674/how-to-check-if-pytorch-is-using-the-gpu
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print("GPU used: " + torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Cude available: True
Cude enabled:True
Using device: cuda

GPU used: Tesla K80
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


# Load Data



1. Load CSV
2. Add Technical indicators
3. Create TabList

In [0]:
# Set path
path = base_dir
path

'/content/gdrive/My Drive/fastai-v3/'

In [0]:
!wget https://raw.githubusercontent.com/marvin-hansen/fastai/master/AMZN.csv

In [52]:
file_name="AMZN.csv"
df = pd.read_csv(file_name)
df.tail(1)


,Date,Open,High,Low,Close,Adj Close,Volume
5452,2019-03-01,1655.130005,1674.26001,1651.0,1671.72998,1671.72998,4974900


In [0]:
dep_var = 'Close'
cont_names = ['Open', 'High', 'Low', 'Adj Close', 'Volume']

procs = [FillMissing, Normalize]


In [0]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cont_names=cont_names)


In [0]:
data = (TabularList.from_df(df, path=path, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [56]:
data.show_batch(rows=3)


Open,High,Low,Adj Close,Volume,target
-0.4885,-0.4868,-0.4888,-0.4851,-0.0656,74.53
-0.4236,-0.4247,-0.4416,-0.4316,1.5692,96.75
0.9668,0.9941,0.9870,1.0062,-0.2630,693.97


## Functions

In [0]:
# Fixes the following error: Expected object of scalar type Long but got scalar type Float for argument #2 ‘other
# https://forums.fast.ai/t/error-pytorch-expected-object-of-scalar-type-long-but-got-scalar-type-float-for-argument-2-other/33778
def accuracy_long(input:Tensor, targs:Tensor)->Rank0Tensor:
    "Compute accuracy with `targs` when `input` is bs * n_classes."
    n = targs.shape[0]
    input = input.argmax(dim=-1).view(n,-1)
    #targs = targs.view(n,-1)
    targs = targs.view(-1).long()
    return (input==targs).float().mean()

In [0]:
def get_technical_indicators(dataset):
    # Create Moving Averages
    dataset['ma7'] = dataset[5].rolling(window=7).mean()
    dataset['ma21'] = dataset[5].rolling(window=21).mean()
    dataset['ma30'] = dataset[5].rolling(window=30).mean()
    dataset['ma50'] = dataset[5].rolling(window=50).mean()
    dataset['ma200'] = dataset[5].rolling(window=200).mean()
    
    # Create Exponential moving average
    dataset['ema'] = dataset[5].ewm(com=0.5).mean()
    
    # Create MACD
    dataset['26ema'] = pd.ewma(dataset[5], span=26)
    dataset['12ema'] = pd.ewma(dataset[5], span=12)
    dataset['MACD'] = (dataset['12ema']-dataset['26ema'])

    # Create Bollinger Bands
    dataset['20sd'] = pd.stats.moments.rolling_std(dataset[5],20)
    dataset['upper_band'] = dataset['ma21'] + (dataset['20sd']*2)
    dataset['lower_band'] = dataset['ma21'] - (dataset['20sd']*2)
    
    return dataset

## Train model on data

In [0]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy_long)


In [61]:
epochs = 5
lr = 1e-3
learn.fit(epochs, lr)


epoch,train_loss,valid_loss,accuracy_long,time
1,6481.901367,458.382904,0.000000,00:00
2,5907.617676,376.875153,0.000000,00:00
3,5765.332520,605.917480,0.000000,00:00
4,6647.208984,54.400162,0.000000,00:00
5,7220.656738,103.718636,0.000000,00:01
